In [1]:
!nvidia-smi

Sun Jun 19 14:15:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   37C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
import zipfile
import numpy as np
# from imgaug import augmenters as iaa

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [3]:
!wget 'https://storage.googleapis.com/kimata/datasets/face%20mask/dataset_full.zip'

--2022-06-19 09:01:09--  https://storage.googleapis.com/kimata/datasets/face%20mask/dataset_full.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.4.80, 172.217.4.208, 142.250.191.112, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.4.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1198930675 (1.1G) [application/zip]
Saving to: 'dataset_full.zip'

dataset_full.zip    100%[===================>]   1.12G   128MB/s    in 10s     

2022-06-19 09:01:20 (110 MB/s) - 'dataset_full.zip' saved [1198930675/1198930675]



In [4]:
!ls

'Kaggle Feature engineering'   Zindi		  face-mask-detection.ipynb
 Untitled.ipynb		       dataset_full.zip   mnist_pca.npy


In [5]:
zip_ref = zipfile.ZipFile('dataset_full.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [6]:
# #Define image augmentation steps.
# def apply_augmentation(input_image):
#     aug_list = iaa.Sequential([
#         iaa.SaltAndPepper(0.1),
#         iaa.GammaContrast((0.8, 1.0), per_channel = True),
#         iaa.AdditiveGaussianNoise(scale = (0, 0.2 * 255))
#     ])
    
#     image_aug = aug_list(image = input_image)
#     return image_aug

In [3]:
PATH_TO_IMAGES = 'dataset_full'
BATCH_SIZE = 32
CLASS_MODE = 'categorical'

datagen_train = ImageDataGenerator(rescale = 1 / 255,
                                  # preprocessing_function = apply_augmentation,
                                  validation_split = 0.2)


In [4]:
train_generator = datagen_train.flow_from_directory(PATH_TO_IMAGES, 
                                                   target_size = (300, 300),
                                                   batch_size = BATCH_SIZE, 
                                                   class_mode = CLASS_MODE,
                                                   subset = 'training')

test_generator = datagen_train.flow_from_directory(PATH_TO_IMAGES,
                                                 target_size = (300, 300),
                                                 batch_size = BATCH_SIZE,
                                                 class_mode = CLASS_MODE,
                                                 subset = 'validation')

Found 4162 images belonging to 3 classes.
Found 1039 images belonging to 3 classes.


In [9]:
#Use different pretrained models for classification.
class KerasFactory():
    #This class can be used to create a keras model and return it.
    def __init__(self):
        pass
    
    def create_and_return_keras_model(self, model_name, num_classes, input_shape, 
                                     activation_func = 'softmax', transfer_learning = True):
    
        if (not isinstance(model_name, str)):
            raise ValueError('Input model_name has the wrong datatype!')

        if (not isinstance (num_classes, int)):
            raise ValueError('Input num_classes has the wrong datatype.')

        if (not isinstance (transfer_learning, bool)):
            raise ValueError('Input transfer_learning has the wrong datatype.')

        if (not isinstance (activation_func, str)):
            raise ValueError('Input activation_func has the wrong datatype.')

        weights = 'imagenet' if transfer_learning else None

        if model_name == 'mobilenet_v2':
            from tensorflow.keras.applications import MobileNetV2
            model = MobileNetV2(weights = weights, include_top = False, input_shape = input_shape)

        elif model_name == 'mobilenet_v3':
            from tensorflow.keras.applications import MobileNetV3Small
            model = MobileNetV3Small(include_top = False, weights = weights, input_shape = input_shape)
            
        elif model_name == 'densenet_121':
            from tensorflow.keras.applications.densenet import DenseNet121
            model = DenseNet121(include_top = False, weights = weights, input_shape = input_shape)
            
        elif model_name == 'resnet_50':
            from tensorflow.keras.applications.resnet50 import ResNet50 
            model = ResNet50(include_top = False, weights = weights, input_shape = input_shape)
            
        else:
            from classification_models.tfkeras import Classifiers
            net, _ = Classifiers.get(model_name)
            model = net(input_shape = input_shape, include_top = False, weigts = weights)
        
        
        #Add a global spatial averaging pooling layer to decrease the risk of overfitting and plot the class activation maps.
        output = model.output
        x = tf.keras.layers.GlobalAveragePooling2D()(output)

        #Add a layer of dropout to also avoid overfitting.
        x = tf.keras.layers.Dropout(0.3)(x)

        #Add a fully connected layer.
        x = tf.keras.layers.Dense(128, activation = 'relu')(x)

        #Add another layer of dropout.
        x = tf.keras.layers.Dropout(0.3)(x)

        #Output layer.
        predictions = tf.keras.layers.Dense(num_classes, activation = activation_func)(x)

        #Create the model.
        model = tf.keras.Model(inputs = model.input, outputs = predictions)

        return model

In [10]:
#Test keras factory.
factory = KerasFactory()

In [11]:
devices = tf.config.list_physical_devices()
devices

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [12]:
encoder_list = ['densenet_121', 'resnet_50', 'mobilenet_v2', 'mobilenet_v3']
num_epochs = 30

for encoder in encoder_list:
    print(f'Start training {encoder}...')
    model = factory.create_and_return_keras_model(encoder, 
                                                 num_classes = 3,
                                                 input_shape = (800, 600, 3),
                                                 activation_func = 'softmax',
                                                 transfer_learning = True)
    
    opt = tf.keras.optimizers.Adam(learning_rate = 1e-4)
    
    #Define metrics.
    metrics_list = [
        tf.keras.metrics.CategoricalAccuracy(name = 'accuracy'),
        tf.keras.metrics.AUC(name = 'auc'),
        tfa.metrics.F1Score(num_classes = 3, average = 'weighted', threshold = 0.5)
    ]
    
    #Tensorboard callback.
    tb_callback = TensorBoard(log_dir = f'logs/{encoder}')
    
    #Compile model.
    model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = metrics_list)
    
    #Fit the model.
    with tf.device('GPU:0'):
        model.fit(train_generator, epochs = num_epochs, verbose = 1,validation_data = test_generator, callbacks = [tb_callback])

Start training densenet_121...
29084464/29084464 [==============================] - 0s 0us/step
Epoch 1/30
131/131 [==============================] - 120s 760ms/step - loss: 0.1884 - accuracy: 0.9270 - auc: 0.9902 - f1_score: 0.9282 - val_loss: 0.1961 - val_accuracy: 0.9211 - val_auc: 0.9891 - val_f1_score: 0.9235
Epoch 2/30
131/131 [==============================] - 91s 694ms/step - loss: 0.0511 - accuracy: 0.9827 - auc: 0.9989 - f1_score: 0.9824 - val_loss: 0.1026 - val_accuracy: 0.9702 - val_auc: 0.9949 - val_f1_score: 0.9695
Epoch 3/30
131/131 [==============================] - 92s 701ms/step - loss: 0.0433 - accuracy: 0.9825 - auc: 0.9995 - f1_score: 0.9831 - val_loss: 0.0564 - val_accuracy: 0.9808 - val_auc: 0.9985 - val_f1_score: 0.9805
Epoch 4/30
131/131 [==============================] - 93s 706ms/step - loss: 0.0352 - accuracy: 0.9887 - auc: 0.9995 - f1_score: 0.9888 - val_loss: 0.0230 - val_accuracy: 0.9904 - val_auc: 0.9994 - val_f1_score: 0.9903
Epoch 5/30
131/131 [=======

In [12]:
#Compress the logs file and download it.
import shutil
shutil.make_archive('logs_full', 'zip', 'logs/')

'/home/studio-lab-user/sagemaker-studiolab-notebooks/Structured data/logs_full.zip'